# Exercice d'économétrie

## Netoyage des données

### Modules a installer

In [1]:
!pip install wbdata
!pip install country-converter
!pip install igraph

### Importation 

In [2]:
#Module pour la gestion de l'environnement de travail
import os

#Module pour la manipulation des données
import pandas as pd

#Module pour se connecter à l'API des bases de données de la banque mondiale
import wbdata

#Module pour manipulation de date
import datetime

#Module pour la conversion des ISO codes
import country_converter as coco

#Module pour manipulation de matice et calcule
import numpy as np

### Définition de l'environnement de travail

In [3]:
Paths = {"richm"    : "C://Users//richm//Dropbox//Projet d'économétrie",
         "frans"    : "C://Users//frans//Dropbox//Projet d'économétrie",
         "richa"    : "C://Users//richa//Dropbox//Projet d'économétrie",
         "rgnikiea" : "//Users//rgnikiea//Dropbox//Projet d'économétrie"}

os.chdir(Paths[os.getlogin()])

### Liste des indicateur a extraire du WDI

In [5]:
indicators = {
              "NY.GDP.MKTP.KD": "GDP", # Gross Domestic Product
              "BX.KLT.DINV.WD.GD.ZS":"FDI",
              "BX.TRF.PWKR.DT.GD.ZS":"Remit(% of GDP)",
              "NY.GDP.MKTP.KD.ZG":"GDP growth (annual %)",
              "FX.OWN.TOTL.MA.ZS":"own_acount ", #Account ownership at a financial institution or with a mobile-money-service provider, male (% of population ages 15+)
              "PX.REX.REER":"TCR" , #Real effective exchange rate index (2010 = 100)
              "FR.INR.LEND":"lend_rate", # Lending interest rate (%)
              "FM.LBL.BMNY.GD.ZS": "M2", # Broad money (% of GDP)
              "FS.AST.PRVT.GD.ZS" : "Credit_priv", # Credit to private sector 
              "NE.TRD.GNFS.ZS" : "TRADE", #Trade % PIB
}

### Imporation de données contenant les noms des pays en developpement, leur régions et leurs groupe de revenu

In [6]:
df = pd.read_excel("02. Data/Raw data//Countries_ids.xlsx")
df = df[df['Income Group'].isin(['Low income', 'Lower middle income', 'Upper middle income'])]
df.drop(columns = "Country name", axis = 1, inplace = True)
df.rename(columns = {'ISO 3166-1 alpha-3 country code':'iso'}, inplace=True)
df.head()

,iso,Region,Income Group
0,AFG,South Asia,Low income
1,ALB,Europe & Central Asia,Upper middle income
2,DZA,Middle East & North Africa,Lower middle income
3,ASM,East Asia & Pacific,Upper middle income
5,AGO,Sub-Saharan Africa,Lower middle income


In [7]:
maps = {"KSV":"XKX", "TMP":"TLS"}
df["iso"] = df.iso.replace(maps)
iso = list(df.iloc[:,0])

In [8]:
df["Income Group"].unique()

array(['Low income', 'Upper middle income', 'Lower middle income'],
      dtype=object)

In [9]:
df["iso"].nunique()

137

### Configuration de la requete et extraction des données du WDI

In [10]:
data_date = datetime.datetime(1990, 1, 1), datetime.datetime(2022, 1,1)

In [11]:
df2 = wbdata.get_dataframe(indicators, country = iso, data_date=data_date)
df2.reset_index(inplace = True)
df2.head()

,country,date,GDP,FDI,Remit(% of GDP),GDP growth (annual %),own_acount,TCR,lend_rate,M2,Credit_priv,TRADE
0,Afghanistan,2021,1.708357e+10,0.139319,NaN,-20.737146,14.79,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2020,2.155306e+10,0.064389,3.916496,-2.351101,NaN,NaN,NaN,37.209463,3.071936,NaN
2,Afghanistan,2019,2.207200e+10,0.123804,4.382937,3.911603,NaN,NaN,NaN,34.959214,3.229957,NaN
3,Afghanistan,2018,2.124113e+10,0.648440,4.362631,1.189228,NaN,NaN,NaN,36.607494,3.512317,NaN
4,Afghanistan,2017,2.099149e+10,0.272719,4.353918,2.647003,22.54,NaN,14.836667,36.861261,3.841881,NaN


### Création des ISO code

In [12]:
cntries = df2["country"]
ISO_3 = coco.convert(names=cntries, to='ISO3')
df2["iso"] = ISO_3

### Fusion des données du WDI et des données sur les groupes de revenus et des régions

In [13]:
data = df2.merge(df, on = ["iso"])
data.rename(columns = {"date":"year"}, inplace = True)
data["year"] = data.year.astype("int")
display(data.info())
data.head()

<class 'wbdata.api.WBDataFrame'>
Int64Index: 4384 entries, 0 to 4383
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                4384 non-null   object 
 1   year                   4384 non-null   int32  
 2   GDP                    4149 non-null   float64
 3   FDI                    3995 non-null   float64
 4   Remit(% of GDP)        3617 non-null   float64
 5   GDP growth (annual %)  4124 non-null   float64
 6   own_acount             373 non-null    float64
 7   TCR                    1604 non-null   float64
 8   lend_rate              2723 non-null   float64
 9   M2                     3650 non-null   float64
 10  Credit_priv            3140 non-null   float64
 11  TRADE                  3673 non-null   float64
 12  iso                    4384 non-null   object 
 13  Region                 4384 non-null   object 
 14  Income Group           4384 non-null   object 
dtypes: float6

None

,country,year,GDP,FDI,Remit(% of GDP),GDP growth (annual %),own_acount,TCR,lend_rate,M2,Credit_priv,TRADE,iso,Region,Income Group
0,Afghanistan,2021,1.708357e+10,0.139319,NaN,-20.737146,14.79,NaN,NaN,NaN,NaN,NaN,AFG,South Asia,Low income
1,Afghanistan,2020,2.155306e+10,0.064389,3.916496,-2.351101,NaN,NaN,NaN,37.209463,3.071936,NaN,AFG,South Asia,Low income
2,Afghanistan,2019,2.207200e+10,0.123804,4.382937,3.911603,NaN,NaN,NaN,34.959214,3.229957,NaN,AFG,South Asia,Low income
3,Afghanistan,2018,2.124113e+10,0.648440,4.362631,1.189228,NaN,NaN,NaN,36.607494,3.512317,NaN,AFG,South Asia,Low income
4,Afghanistan,2017,2.099149e+10,0.272719,4.353918,2.647003,22.54,NaN,14.836667,36.861261,3.841881,NaN,AFG,South Asia,Low income


### Ajout des données sur la stabilité du gouvernement 

In [14]:
df = pd.read_excel("02. Data/Raw data/icrg_database.xlsx")
df.rename(columns = {"ccode":"iso","A-Government Stability":"gov_stab",
                    'F-Corruption':'corup'}, inplace=True)
df = df[["iso", "year" , "gov_stab", "corup"]]
df.head()

,iso,year,gov_stab,corup
0,ALB,1984,4.833333,4
1,ALB,1985,6.666667,4
2,ALB,1986,7,4
3,ALB,1987,7,4
4,ALB,1988,7,4


In [15]:
data = data.merge(df, on = ["iso", "year"], how = "left")
display(data.info())
data.head()

<class 'wbdata.api.WBDataFrame'>
Int64Index: 4384 entries, 0 to 4383
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                4384 non-null   object 
 1   year                   4384 non-null   int32  
 2   GDP                    4149 non-null   float64
 3   FDI                    3995 non-null   float64
 4   Remit(% of GDP)        3617 non-null   float64
 5   GDP growth (annual %)  4124 non-null   float64
 6   own_acount             373 non-null    float64
 7   TCR                    1604 non-null   float64
 8   lend_rate              2723 non-null   float64
 9   M2                     3650 non-null   float64
 10  Credit_priv            3140 non-null   float64
 11  TRADE                  3673 non-null   float64
 12  iso                    4384 non-null   object 
 13  Region                 4384 non-null   object 
 14  Income Group           4384 non-null   object 
 15  gov_stab 

### Ajout des données sur l'inflation

In [16]:
df = pd.read_excel("02. Data/Raw data/weo_database_inflation.xlsx")
df.drop(columns =  ['WEO Country Code', 'WEO Subject Code', 'Country','Subject Descriptor', 'Subject Notes', 'Units', 'Scale'], inplace =True )
df = df.melt(id_vars = 'ISO', var_name = "year", value_name= "inf")
df["year"] = df["year"].str.replace("y","").astype("int")
df["inf"]  = pd.to_numeric(df['inf'], errors='coerce')
df.rename(columns = {"ISO":"iso"}, inplace = True)
df.head()

,iso,year,inf
0,AFG,1980,NaN
1,ALB,1980,NaN
2,DZA,1980,9.668
3,AND,1980,NaN
4,AGO,1980,46.708


In [17]:
data = data.merge(df, on = ["iso", "year"], how = "left")
display(data.info())
data.head()

<class 'wbdata.api.WBDataFrame'>
Int64Index: 4384 entries, 0 to 4383
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                4384 non-null   object 
 1   year                   4384 non-null   int32  
 2   GDP                    4149 non-null   float64
 3   FDI                    3995 non-null   float64
 4   Remit(% of GDP)        3617 non-null   float64
 5   GDP growth (annual %)  4124 non-null   float64
 6   own_acount             373 non-null    float64
 7   TCR                    1604 non-null   float64
 8   lend_rate              2723 non-null   float64
 9   M2                     3650 non-null   float64
 10  Credit_priv            3140 non-null   float64
 11  TRADE                  3673 non-null   float64
 12  iso                    4384 non-null   object 
 13  Region                 4384 non-null   object 
 14  Income Group           4384 non-null   object 
 15  gov_stab 

### Ajout des données sur l'ouverture du capital

In [20]:
df = pd.read_stata("02. Data/Raw data/capital_openess_database.dta")
df.rename(columns = {"ccode":"iso"}, inplace = True)
df  = df[["iso", "year", "ka_open"]]
df.head()

,iso,year,ka_open
0,USA,1970,1.0
1,USA,1971,1.0
2,USA,1972,1.0
3,USA,1973,1.0
4,USA,1974,1.0


In [21]:
data = data.merge(df, on = ["iso", "year"], how = "left")
data.info()

<class 'wbdata.api.WBDataFrame'>
Int64Index: 4384 entries, 0 to 4383
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                4384 non-null   object 
 1   year                   4384 non-null   int32  
 2   GDP                    4149 non-null   float64
 3   FDI                    3995 non-null   float64
 4   Remit(% of GDP)        3617 non-null   float64
 5   GDP growth (annual %)  4124 non-null   float64
 6   own_acount             373 non-null    float64
 7   TCR                    1604 non-null   float64
 8   lend_rate              2723 non-null   float64
 9   M2                     3650 non-null   float64
 10  Credit_priv            3140 non-null   float64
 11  TRADE                  3673 non-null   float64
 12  iso                    4384 non-null   object 
 13  Region                 4384 non-null   object 
 14  Income Group           4384 non-null   object 
 15  gov_stab 

### Ajout des données sur le developpement financier

In [22]:
df = pd.read_excel("02. Data/Raw data/FD_Index_Database.xlsx")
df = df[["code", "year", "FD"]]
df.rename(columns = {"code":"iso"}, inplace = True)
df.head()

,iso,year,FD
0,ABW,2020,0.291232
1,ABW,2019,0.289913
2,ABW,2018,0.293785
3,ABW,2017,0.294828
4,ABW,2016,0.298656


In [23]:
data = data.merge(df, on = ["iso", "year"], how = "left")
data.info()

<class 'wbdata.api.WBDataFrame'>
Int64Index: 4384 entries, 0 to 4383
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                4384 non-null   object 
 1   year                   4384 non-null   int32  
 2   GDP                    4149 non-null   float64
 3   FDI                    3995 non-null   float64
 4   Remit(% of GDP)        3617 non-null   float64
 5   GDP growth (annual %)  4124 non-null   float64
 6   own_acount             373 non-null    float64
 7   TCR                    1604 non-null   float64
 8   lend_rate              2723 non-null   float64
 9   M2                     3650 non-null   float64
 10  Credit_priv            3140 non-null   float64
 11  TRADE                  3673 non-null   float64
 12  iso                    4384 non-null   object 
 13  Region                 4384 non-null   object 
 14  Income Group           4384 non-null   object 
 15  gov_stab 

### Enregistrement des données

In [25]:
data.columns = data.columns.str.lower()

In [32]:
file_name = "02. Data/Cleaned data/Database.xlsx" 
data.to_excel(file_name)

In [33]:
file_name = "02. Data/Cleaned data/Database.csv" 
data.to_csv(file_name)